In [1]:
import time
import pandas as pd
from sqlalchemy import create_engine
import sqlalchemy
from tqdm import tqdm
import gspread
from oauth2client.service_account import ServiceAccountCredentials


## Variables

In [2]:
# Dates of the campaign
dates = ['2023-01-19', '2023-02-17']
# Cluster dictionary
cluster_names = pd.read_excel("/Users/gabrielreynoso/Documents/RFM_LayerA_Dic.xlsx")
# Create the channel dictionary
cluster_dict = cluster_names.set_index('id').to_dict()['name']
# BD Connection
f = open('/Users/gabrielreynoso/Documents/Queries/db_klarprod_connection.txt', 'r')
postgres_str = f.read()
f.close()
cnx = create_engine(postgres_str)
# Read and Load Credentials
credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/gabrielreynoso/Documents/GoogleCredentials/gabo_credentials.json')
gc = gspread.authorize(credentials)

## Queries

In [56]:
# RFM Query for data on a specific date
query_rfm = '''
        select rfm_layer.user_id,
               max_user.max_date as date,
               rfm_layer.cluster,
               rfm_layer.time,
               rfm_layer.monetary,
               rfm_layer.frequency,
               rfm_layer.recency
        from growth.rfm_history as rfm_layer,
             (select user_id,
                     max(date) as max_date
             from growth.rfm_history
             where date < '{}'::date
             group by user_id) max_user
        where rfm_layer.user_id = max_user.user_id
        and rfm_layer.date = max_user.max_date
        and rfm_layer.user_id in {}
'''

In [39]:
cohort_query = '''
select
    klar_user_id as user_id,
    segment_name,
    case when segment_name like '%MAR%' then 'More at risk'
         when segment_name like '%More at risk%' then 'More at risk' else 'Churned' end as cohort,
    case when segment_name like '%Trigger%' then 'Trigger'
         when segment_name like '%Treatment' then 'Treatment' else 'Control' end as segment
from is_customer_io.segments
where segment_name like '%RFM January%'
and segment_name not like '%Twilio%'
and cio_id != '';
'''

# Cohort Processing

In [27]:
# Read the Query
cohort_info = pd.read_sql_query(sqlalchemy.text(cohort_query),cnx)

In [28]:
# January Processing
# Count number of segments per user
segments_per_user = cohort_info.groupby('user_id')['segment_name'].count()
# Create variable to count them
segments_per_user = segments_per_user.to_frame('num_segments')
# Add new variable to the data
cohort_info = pd.merge(segments_per_user, cohort_info, on='user_id')
# Filter the users within 2 segments
cohort_info = cohort_info[cohort_info.num_segments == 2]
# Extract the subsegment
cohort_info['subsegment'] = cohort_info.segment_name.str.extract("(\d+ pesos)")
# Filter the rows with the segment info
cohort_info_segment= cohort_info[cohort_info.subsegment.isna()][['user_id','cohort','segment']]
# Filter the rows with the subsegment info
cohort_info_subsegment= cohort_info[~cohort_info.subsegment.isna()][['user_id','cohort','subsegment']]
# Do an inner join to get the users that have the complete info
cohort_info = pd.merge(cohort_info_subsegment, cohort_info_segment, how = 'inner', on = ['user_id', 'cohort'])

In [29]:
# Print the final result
cohort_info

,user_id,cohort,subsegment,segment
0,000017e5-eeb7-49b5-836b-189f64fb21f6,Churned,50 pesos,Control
1,00006457-7797-46e8-87bf-7cf88f93f7a3,Churned,50 pesos,Treatment
2,000162b9-0297-435e-ae32-375c7fe1a082,Churned,100 pesos,Control
3,0001be16-8430-4314-bac4-f23119645859,More at risk,100 pesos,Treatment
4,0001e46b-bade-479e-967f-0481ed7318cb,More at risk,50 pesos,Treatment
...,...,...,...,...
162283,fffe0258-489b-46b8-b993-5e89216ce91c,More at risk,50 pesos,Treatment
162284,fffe9782-847e-4b40-a37b-b9784396c772,More at risk,100 pesos,Treatment
162285,fffefd49-2f4b-45ae-b235-9fb777865752,More at risk,50 pesos,Treatment
162286,ffffa15a-5b9e-44fa-8725-a0e844c103d6,More at risk,50 pesos,Treatment


In [47]:
cohort_info[cohort_info.user_id == 'ee7f3e32-9b95-4f28-8b84-50dbefa6fae0']

,user_id,cohort,subsegment,segment
151052,ee7f3e32-9b95-4f28-8b84-50dbefa6fae0,More at risk,300 pesos,Treatment


# Run queries info

## Start of the campaign

In [30]:
start_time = time.time()
# Start of the campaign
start_campaign = pd.read_sql_query(query_rfm.format(dates[0], tuple(cohort_info.user_id.to_list())), cnx)
print(time.time()-start_time)

40.31041502952576


In [31]:
start_campaign = start_campaign.set_index('user_id')
start_campaign['cluster'] = start_campaign['cluster'].map(cluster_dict)
start_campaign = start_campaign.add_prefix('Pre_')

In [32]:
start_campaign

,Pre_date,Pre_cluster,Pre_time,Pre_monetary,Pre_frequency,Pre_recency
user_id,,,,,,
fdb66bc4-a6d2-4815-ac89-506ac9c1b7d7,2023-01-18,At Risk,193.0,3.34,0.021,155.0
48ae96e3-ccd0-4b94-84ae-ab280befa661,2023-01-18,At Risk,499.0,6.26,0.082,304.0
fdbaf1c9-3742-49a9-a8d5-94ba4feeb036,2023-01-18,Churned,332.0,0.27,0.012,317.0
48b57682-f148-4861-b5ee-12e00ed1e6f8,2023-01-18,At Risk,377.0,0.86,0.013,241.0
fdc233bd-a16b-4c5d-8dce-ad9b675747c3,2023-01-18,At Risk,312.0,4.46,0.054,226.0
...,...,...,...,...,...,...
6fda059b-3eb5-48cf-8011-1da0174b5498,2023-01-18,Churned,364.0,1.95,0.022,336.0
6fe1d451-8f7f-4008-b079-95ba814a3cce,2023-01-18,At Risk,283.0,0.35,0.021,227.0
6fe37ff6-f7a4-4904-b8b6-b0d270048200,2023-01-18,Churned,341.0,1.89,0.029,323.0


## End of the campaign

In [33]:
start_time = time.time()
# Start of the campaign
end_campaign = pd.read_sql_query(query_rfm.format(dates[1], tuple(cohort_info.user_id.to_list())), cnx)
print(time.time()-start_time)

27.927987337112427


In [34]:
end_campaign = end_campaign.set_index('user_id')
end_campaign['cluster'] = end_campaign['cluster'].map(cluster_dict)
end_campaign = end_campaign.add_prefix('Post_')

In [35]:
end_campaign

,Post_date,Post_cluster,Post_time,Post_monetary,Post_frequency,Post_recency
user_id,,,,,,
00028a82-a5ff-40d3-b21d-6bc9afed1d77,2023-02-16,Churned,424.0,2.15,0.144,313.0
000726e1-1998-4f34-892f-ffa55f28988f,2023-02-16,At Risk,303.0,0.92,0.050,262.0
0017ca6e-2e8d-46cd-abd7-8dd98f332d4a,2023-02-16,At Risk,155.0,0.58,0.006,155.0
e7e15eaf-0e7a-482b-9449-fe3ee7483073,2023-02-16,At Risk,508.0,1.81,0.010,280.0
001b67c1-8c44-471d-b464-1dfe9ca4d85a,2023-02-16,At Risk,175.0,0.23,0.011,175.0
...,...,...,...,...,...,...
4f1f06d7-3727-42c6-82f3-30fe9ffc88f8,2023-02-16,Churned,332.0,0.15,0.006,303.0
4f210737-df98-4b06-8d89-a6b4bf232430,2023-02-16,At Risk,255.0,0.82,0.012,237.0
4f2bd78b-348e-4d8e-b0b1-898667f8438d,2023-02-16,Churned,385.0,0.21,0.005,333.0


In [36]:
# Merge Pre and Post campaign
rfm_info = pd.merge(start_campaign, end_campaign, left_index=True, right_index=True)
rfm_info = rfm_info.reset_index()
rfm_info['movement'] = rfm_info['Pre_cluster'] + '->' + rfm_info['Post_cluster']
rfm_info

,user_id,Pre_date,Pre_cluster,Pre_time,Pre_monetary,Pre_frequency,Pre_recency,Post_date,Post_cluster,Post_time,Post_monetary,Post_frequency,Post_recency,movement
0,fdb66bc4-a6d2-4815-ac89-506ac9c1b7d7,2023-01-18,At Risk,193.0,3.34,0.021,155.0,2023-02-16,At Risk,222.0,2.90,0.018,184.0,At Risk->At Risk
1,48ae96e3-ccd0-4b94-84ae-ab280befa661,2023-01-18,At Risk,499.0,6.26,0.082,304.0,2023-02-16,At Risk,528.0,5.92,0.078,333.0,At Risk->At Risk
2,fdbaf1c9-3742-49a9-a8d5-94ba4feeb036,2023-01-18,Churned,332.0,0.27,0.012,317.0,2023-02-16,Churned,361.0,0.25,0.011,346.0,Churned->Churned
3,48b57682-f148-4861-b5ee-12e00ed1e6f8,2023-01-18,At Risk,377.0,0.86,0.013,241.0,2023-02-16,Churned,406.0,0.80,0.012,270.0,At Risk->Churned
4,fdc233bd-a16b-4c5d-8dce-ad9b675747c3,2023-01-18,At Risk,312.0,4.46,0.054,226.0,2023-02-16,At Risk,341.0,4.08,0.050,255.0,At Risk->At Risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162281,6fda059b-3eb5-48cf-8011-1da0174b5498,2023-01-18,Churned,364.0,1.95,0.022,336.0,2023-02-16,Churned,393.0,1.80,0.020,365.0,Churned->Churned
162282,6fe1d451-8f7f-4008-b079-95ba814a3cce,2023-01-18,At Risk,283.0,0.35,0.021,227.0,2023-02-16,At Risk,312.0,0.32,0.019,256.0,At Risk->At Risk
162283,6fe37ff6-f7a4-4904-b8b6-b0d270048200,2023-01-18,Churned,341.0,1.89,0.029,323.0,2023-02-16,Churned,370.0,1.74,0.027,352.0,Churned->Churned
162284,6fefbc13-2b74-4157-aec1-3c6e11fed16c,2023-01-18,Churned,331.0,0.15,0.003,331.0,2023-02-16,Churned,360.0,0.14,0.003,360.0,Churned->Churned


## Purchases

In [66]:
# Purchases for the users in the specific date
query_p = '''
        select
               t.user_id,
               count(transaction_id) as purchases,
                -1*sum(amount) as total_amount
        from analytics_bi.transactions as t
        where t.type in ('PURCHASE')
          and t.timestamp_mx_created_at between '{}'::date and '{}'::date
          and t.state = 'SETTLED'
          and t.source_account_internal_id <> '0000000000000000'
          and t.source_account_internal_id <> '00000000-0000-0000-0000-000000000000'
          and t.provider_id <> 'KLAR'
          and t.user_id in {}
        group by t.user_id
'''

In [70]:
# Purchases
start_time = time.time()
query_purchases = query_p.format(dates[0],dates[1], tuple(cohort_info.user_id.to_list()))
purchases = pd.read_sql_query(query_purchases, cnx)
print(time.time()-start_time)

45.59776997566223


In [71]:
purchases

,user_id,purchases,total_amount
0,ed922324-d303-4c9c-89db-5f74a81eaadc,2,174.00
1,ee7f3e32-9b95-4f28-8b84-50dbefa6fae0,1,179.00
2,eeab229a-9e77-49d3-93da-3f450c166657,2,777.00
3,ef04d072-19b5-404e-97bd-6fb77d22e4b0,3,1495.00
4,ef522dbd-2f9d-4aaa-919b-181a797af4eb,1,400.00
...,...,...,...
4220,de3511d5-3a28-4464-93a9-b98d087ab92f,2,408.00
4221,de4c9d02-f0b1-4c8c-843a-4578990b22fb,1,60.00
4222,e4b843e6-177f-4d5f-a1df-c87a646174cf,1,70.34
4223,e701ebe3-9858-4f23-9911-0a7d7ca1edc6,1,1.00


# Add info

In [83]:
complete_info = pd.merge(cohort_info, rfm_info, on='user_id')
complete_info = pd.merge(complete_info, purchases, on='user_id', how='left')
complete_info

,user_id,cohort,subsegment,segment,Pre_date,Pre_cluster,Pre_time,Pre_monetary,Pre_frequency,Pre_recency,Post_date,Post_cluster,Post_time,Post_monetary,Post_frequency,Post_recency,movement,purchases,total_amount
0,000017e5-eeb7-49b5-836b-189f64fb21f6,Churned,50 pesos,Control,2023-01-18,Churned,389.0,0.44,0.015,335.0,2023-02-16,Churned,418.0,0.41,0.014,364.0,Churned->Churned,NaN,NaN
1,00006457-7797-46e8-87bf-7cf88f93f7a3,Churned,50 pesos,Treatment,2023-01-18,Churned,319.0,0.67,0.016,293.0,2023-02-16,Churned,348.0,0.61,0.014,322.0,Churned->Churned,NaN,NaN
2,000162b9-0297-435e-ae32-375c7fe1a082,Churned,100 pesos,Control,2023-01-18,At Risk,665.0,4.88,0.041,331.0,2023-02-16,At Risk,694.0,4.67,0.039,360.0,At Risk->At Risk,NaN,NaN
3,0001be16-8430-4314-bac4-f23119645859,More at risk,100 pesos,Treatment,2023-01-18,At Risk,205.0,1.32,0.015,197.0,2023-02-16,At Risk,234.0,1.16,0.013,226.0,At Risk->At Risk,NaN,NaN
4,0001e46b-bade-479e-967f-0481ed7318cb,More at risk,50 pesos,Treatment,2023-01-18,At Risk,181.0,0.83,0.017,181.0,2023-02-16,At Risk,210.0,0.71,0.014,210.0,At Risk->At Risk,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162281,fffe0258-489b-46b8-b993-5e89216ce91c,More at risk,50 pesos,Treatment,2023-01-18,At Risk,244.0,0.41,0.016,196.0,2023-02-16,At Risk,273.0,0.37,0.015,225.0,At Risk->At Risk,NaN,NaN
162282,fffe9782-847e-4b40-a37b-b9784396c772,More at risk,100 pesos,Treatment,2023-01-18,At Risk,204.0,0.98,0.015,184.0,2023-02-16,At Risk,233.0,0.86,0.013,213.0,At Risk->At Risk,NaN,NaN
162283,fffefd49-2f4b-45ae-b235-9fb777865752,More at risk,50 pesos,Treatment,2023-01-18,At Risk,146.0,0.42,0.014,137.0,2023-02-16,Engaged,175.0,0.35,0.017,21.0,At Risk->Engaged,1.0,0.0
162284,ffffa15a-5b9e-44fa-8725-a0e844c103d6,More at risk,50 pesos,Treatment,2023-01-18,At Risk,263.0,0.30,0.011,258.0,2023-02-16,At Risk,292.0,0.27,0.010,287.0,At Risk->At Risk,NaN,NaN


# Cohorts

In [100]:
# Export results
writer = pd.ExcelWriter('./Results/January2023_ChurnCampaign_Results.xlsx', engine='xlsxwriter')
# For loop for extracting cohort information
groups = {}
groups_results = {}
groups_movement = {}
# List with the cohort names
cohort_names = complete_info.cohort.unique().tolist()
for idx, cohort in enumerate(cohort_names):
    # Get cohort info
    aux_cohort = complete_info[complete_info.cohort == cohort]
    # List segments
    segments = aux_cohort.segment.unique().tolist()
    start_time = time.time()
    j=0
    for segment in tqdm(segments):
        print(segment)
        # Filter segment
        aux_segment = aux_cohort[aux_cohort.segment == segment]
        # List segments
        subsegments = aux_segment.subsegment.unique().tolist()
        sheet_name = cohort + '-' + segment
        i = 0
        for subsegment in subsegments:
            # Filter subsegment
            aux_subsegment = aux_segment[aux_segment.subsegment == subsegment]
            # Add to dict of groups the information
            group_name = cohort + '-' + segment[0] + '-' + subsegment
            # Add group dataframe
            groups[group_name] = aux_subsegment
            groups_results[group_name] = aux_subsegment.groupby('movement')[['purchases','total_amount','Pre_monetary','Pre_frequency', 'Pre_recency','Post_monetary', 'Post_frequency', 'Post_recency']].describe()
            groups_movement[group_name] = pd.concat([aux_subsegment.movement.value_counts(), aux_subsegment.movement.value_counts(normalize=True).mul(100)],axis=1, keys=('counts','percentage'))
            # Write the results into an excel
            groups_results[group_name].to_excel(writer, startrow=i+3, startcol=0, sheet_name=sheet_name)
            groups_movement[group_name].to_excel(writer, startrow=i+7+groups_results[group_name].shape[0], startcol=0, sheet_name=sheet_name)
            worksheet = writer.sheets[sheet_name]
            worksheet.write(i, 0, subsegment)
            i = i+9+groups_movement[group_name].shape[0]+groups_results[group_name].shape[0]
        # Cohort name
        sheet_name_segment = cohort + '-' + segment + '-Overall'
        print(sheet_name_segment)
        # Add group dataframe
        groups[sheet_name_segment] = aux_segment
        groups_results[sheet_name_segment] = aux_segment.groupby('movement')[['purchases','total_amount','Pre_monetary','Pre_frequency', 'Pre_recency','Post_monetary', 'Post_frequency', 'Post_recency']].describe()
        groups_movement[sheet_name_segment] = pd.concat([aux_segment.movement.value_counts(), aux_segment.movement.value_counts(normalize=True).mul(100)],axis=1, keys=('counts','percentage'))
        # Write the results into an excel
        groups_results[sheet_name_segment].to_excel(writer, startrow=j+3, startcol=0, sheet_name=sheet_name_segment)
        groups_movement[sheet_name_segment].to_excel(writer, startrow=j+7+groups_results[sheet_name_segment].shape[0], startcol=0, sheet_name=sheet_name_segment)
        worksheet = writer.sheets[sheet_name_segment]
        worksheet.write(j, 0, sheet_name_segment)
        j = j+9+groups_movement[sheet_name_segment].shape[0]+groups_results[sheet_name_segment].shape[0]
    print(str(time.time()-start_time) + ' seconds')
    # Cohort Overall
for type in ['Treatment','Control']:
    # Get cohort info
    aux_cohort = complete_info[complete_info.segment == type]
    overall_type = 'Overall'+'_'+type
    overall_sheet = type + ' ' + 'Overall'
    groups[overall_type] = aux_cohort
    groups_results[overall_type] = aux_cohort.groupby('movement')[['purchases','total_amount','Pre_monetary','Pre_frequency', 'Pre_recency','Post_monetary', 'Post_frequency', 'Post_recency']].describe()
    groups_movement[overall_type] = pd.concat([aux_cohort.movement.value_counts(), aux_cohort.movement.value_counts(normalize=True).mul(100)], axis=1, keys=('counts', 'percentage'))
    # Print the overall results table into excel
    groups_results[overall_type].to_excel(writer, startrow=3, startcol=0, sheet_name=overall_sheet)
    groups_movement[overall_type].to_excel(writer, startrow=7+groups_results[overall_type].shape[0], startcol=0, sheet_name=overall_sheet)
    worksheet = writer.sheets[overall_sheet]
    worksheet.write(0, 0, overall_type)
for type in ['More at risk','Churned']:
    # Get cohort info
    aux_cohort = complete_info[complete_info.cohort == type]
    overall_type = 'Overall'+'_'+type
    overall_sheet = type + ' ' + 'Overall'
    groups[overall_type] = aux_cohort
    groups_results[overall_type] = aux_cohort.groupby('movement')[['purchases','total_amount','Pre_monetary','Pre_frequency', 'Pre_recency','Post_monetary', 'Post_frequency', 'Post_recency']].describe()
    groups_movement[overall_type] = pd.concat([aux_cohort.movement.value_counts(), aux_cohort.movement.value_counts(normalize=True).mul(100)], axis=1, keys=('counts', 'percentage'))
    # Print the overall results table into excel
    groups_results[overall_type].to_excel(writer, startrow=3, startcol=0, sheet_name=overall_sheet)
    groups_movement[overall_type].to_excel(writer, startrow=7+groups_results[overall_type].shape[0], startcol=0, sheet_name=overall_sheet)
    worksheet = writer.sheets[overall_sheet]
    worksheet.write(0, 0, overall_type)
writer.save()

  0%|          | 0/2 [00:00<?, ?it/s]

Control


 50%|█████     | 1/2 [00:00<00:00,  2.65it/s]

Churned-Control-Overall
Treatment


100%|██████████| 2/2 [00:00<00:00,  3.04it/s]


...

 50%|█████     | 1/2 [00:00<00:00,  2.70it/s]

More at risk-Treatment-Overall
Control


100%|██████████| 2/2 [00:00<00:00,  3.03it/s]

More at risk-Control-Overall
0.6617698669433594 seconds



/var/folders/b_/lql2bh7s2797fh2q6lw2ttdh0000gn/T/ipykernel_23776/2965024956.py:80: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
